In [1]:
import hashlib
import random

In [2]:
def mod_exp(base, exp, mod):
    result = 1
    base = base % mod
    while exp > 0:
        if exp % 2 == 1:
            result = (result * base) % mod
        exp = exp >> 1
        base = (base * base) % mod
    return result

In [3]:
def gcd(a, b):
    while b:
        a, b = b, a % b
    return a

In [4]:
def generate_large_prime(bits=1024):
    while True:
        p = random.getrandbits(bits)
        if is_prime(p):
            return p

In [5]:
def is_prime(n, k=128):
    if n == 2 or n == 3:
        return True
    if n % 2 == 0 or n < 2:
        return False
    s, d = 0, n - 1
    while d % 2 == 0:
        s += 1
        d //= 2
    for _ in range(k):
        a = random.randrange(2, n - 1)
        x = pow(a, d, n)
        if x == 1 or x == n - 1:
            continue
        for _ in range(s - 1):
            x = pow(x, 2, n)
            if x == n - 1:
                break
        else:
            return False
    return True

In [6]:
def generate_rsa_keys(p, q):
    n = p * q
    phi_n = (p - 1) * (q - 1)
    
    # e = random.randrange(2, phi_n)
    # while gcd(e, phi_n) != 1:
    #     e = random.randrange(2, phi_n)
    e = 3917625637285712224869106845520840669030227309330572309625012253569441563591905241420372621636051635088862955282825055811360540917488908516960580447922156537062427132930791052831501210298936415421351938056240156338189162115219366681079718135870608493887560653219315575184609206029551759940254176330716274070664478359183820268398751871216818194293348655305182646148864108221552635752245883676553010602132564396156605513699480182637529335133900273687193733228797834949626856276230838592029402712457707039493231481389131994064226818127107580765284314330099061709673424120238209772145363617723054884339188565550441995
    
    d = pow(e, -1, phi_n)
    
    return (e, (n, e)), (d, n)  # Trả về khóa công khai và khóa bí mật

In [7]:
def encrypt_message(message, public_key):
    n, e = public_key
    message_int = int.from_bytes(message.encode(), 'big')  
    encrypted_message = mod_exp(message_int, e, n)  
    return encrypted_message

In [8]:
def hash_message(message):
    hash_object = hashlib.sha256(message.encode())
    return int.from_bytes(hash_object.digest(), 'big')

In [9]:
def sign_message(d, n, message):
    hM = hash_message(message)  
    signature = mod_exp(hM, d, n)
    return signature

In [10]:
def verify_signature(n, e, message, signature):
    print(f"1. Bắt đầu xác minh chữ ký cho thông điệp: '{message}'")
    
    # Bước 1: Băm thông điệp
    hM = hash_message(message)
    print(f"2. Băm thông điệp bằng SHA-256, giá trị băm h(M) = {hM}")
    
    # Bước 2: Giải mã chữ ký bằng khóa công khai (n, e)
    verified = mod_exp(signature, e, n)
    print(f"3. Giải mã chữ ký với e = {e} và n = {n}, thu được giá trị: {verified}")
    
    # Bước 3: So sánh giá trị băm của thông điệp và giá trị đã giải mã
    print(f"4. So sánh giá trị băm h(M) với giá trị đã giải mã:")
    if hM == verified:
        print("5. Chữ ký hợp lệ (h(M) == verified).")
        return True
    else:
        print("5. Chữ ký không hợp lệ (h(M) != verified).")
        return False

In [11]:
p = 27169946228625259547471192772987564682653628818049108614686155180616358671971192192357418636359252855017002811496675719600679817573509624101951464145600845373888915622720022539825922294836177685587286545965224727797986121427289945387953455466785638705211916410795035614355134896529329427593251527948681591127
q = 28614067295713041335834982440918892930920454588097960047059399622908859176792035728565179894899780085826481899938858990689934235759873476495026490062929461947256629510502419808142369636410518283688516348683636240994472917137967158214858615878173964607442222364979676344543373246143706954828683491842492173483
public_key, private_key = generate_rsa_keys(p, q)
print(f"Khóa công khai (n, e): {public_key}")
print(f"Khóa bí mật (d, n): {private_key}")

# Tạo chữ ký
message = "Hoang Kim Chi"  
signature = sign_message(private_key[0], private_key[1], message)
print(f"Chữ ký: {signature}")

# Xác minh chữ ký
# Xác minh chữ ký
if verify_signature(public_key[1][0], public_key[1][1], message, signature):
    print("Chữ ký hợp lệ")
else:
    print("Chữ ký không hợp lệ")

Khóa công khai (n, e): (3917625637285712224869106845520840669030227309330572309625012253569441563591905241420372621636051635088862955282825055811360540917488908516960580447922156537062427132930791052831501210298936415421351938056240156338189162115219366681079718135870608493887560653219315575184609206029551759940254176330716274070664478359183820268398751871216818194293348655305182646148864108221552635752245883676553010602132564396156605513699480182637529335133900273687193733228797834949626856276230838592029402712457707039493231481389131994064226818127107580765284314330099061709673424120238209772145363617723054884339188565550441995, (777442669806787926781605555384246777782063067888725486910873549892122901338196754084683789884316862945464836993279194491719533400737610632684050525233060404226824811601402505029575147654688330856982447308125829275454109411813909814664742839554443038718815281888542747838603435854884582604752469983104079223353759643167268783313550397323254837120583897746765582